Identifying stonewalls

In [2]:
# Local path, change this.
yellow_follow = 'C:/Users/caspe/Desktop/yellow/lib/'

import sys; sys.path.append(yellow_follow) 
import pandas as pd
import ml_utils
import math
import time
import numpy as np
import os

np.set_printoptions(suppress=True)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split

# Tensorflow
import tensorflow_addons as tfa
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Dropout, Conv2D, MaxPooling2D, Flatten, Conv2DTranspose, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.utils import plot_model

folder = "C:/Users/caspe/Desktop/wall_data/"

In [3]:
dtm = np.load(folder + "dtm.npy")
walls = np.load(folder + "walls.npy")
meta = pd.read_csv(folder + "metadata.csv")

In [14]:
walls = (walls == 1).astype('uint8')

In [19]:
np.swapaxes(meta[meta["land"] == 1].values, 0, 1)[0]

array([    4,     5,     6, ..., 19337, 19338, 19339], dtype=int64)

In [21]:
land_mask = np.swapaxes(meta[meta["land"] == 1].values, 0, 1)[0]
wall_mask = np.swapaxes(meta[meta["wall"] == 1].values, 0, 1)[0]    # Has a wall
no_wall_mask = np.swapaxes(meta[meta["wall"] == 2].values, 0, 1)[0] # Has _no_ wall

In [22]:
no_wall_mask.shape

(715,)

In [23]:
wall_mask.shape

(599,)

Regression first

In [25]:
train = np.concatenate([dtm[wall_mask], dtm[no_wall_mask]])
train.shape

(1314, 64, 64)

In [28]:
truth = np.concatenate([meta.values[wall_mask], meta.values[no_wall_mask]])

In [29]:
truth = (np.swapaxes(truth, 0, 1)[2] == 1).astype('uint8')

In [31]:
truth.shape

(1314,)

In [32]:
train = ml_utils.add_rotations(train)
train.shape

(5256, 64, 64)

In [33]:
truth = np.concatenate([truth, truth, truth, truth])

In [35]:
# Shuffle the dataset
shuffle_mask = np.random.permutation(len(truth))
train = train[shuffle_mask]
truth = truth[shuffle_mask]

train = train[..., np.newaxis]

In [37]:
train.shape

(5256, 64, 64, 1)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(train, truth, test_size=0.33, random_state=42)

In [62]:
def define_model(shape, name, activation='relu', kernel_initializer='uniform', maxnorm=4, sizes=[64, 96, 128]):
    model_input = Input(shape=shape, name=name)
    model = Conv2D(sizes[0],
        kernel_size=5,
        padding='same',
        activation=activation,
        kernel_initializer=kernel_initializer,
    )(model_input)

    model = MaxPooling2D(pool_size=(2, 2), padding='same', strides=(2, 2))(model)

    model = Conv2D(sizes[1],
        kernel_size=3,
        padding='same',
        activation=activation,
        kernel_initializer=kernel_initializer,
    )(model)

    model = MaxPooling2D(pool_size=(2, 2), padding='same', strides=(2, 2))(model)

    model = Conv2D(sizes[1],
        kernel_size=3,
        padding='same',
        activation=activation,
        kernel_initializer=kernel_initializer,
    )(model)

    # model = Flatten()(model)

    model = Dense(32, activation='relu', kernel_initializer=kernel_initializer)(model)

    # output = Dense(1, activation='sigmoid')(model)
    output = Conv2D(1, kernel_size=3, padding='same', activation='relu', kernel_initializer=kernel_initializer)(model)

    return Model(inputs=[model_input], outputs=output)

lr = 0.001
bs = 16
epochs = 50

def step_decay(epoch):
    initial_lrate = lr
    drop = 0.5
    epochs_drop = 5
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

model = define_model(X_train.shape[1:], "Generative")

model.compile(
    optimizer=Adam(
        learning_rate=lr,
        name="Adam",
    ),
    loss='BinaryCrossentropy',
)

In [63]:
model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    verbose=1,
    batch_size=bs,
    validation_split=0.3,
    callbacks=[
        LearningRateScheduler(step_decay),
        EarlyStopping(
            monitor="val_loss",
            patience=10,
            min_delta=0.1,
            restore_best_weights=True,
        ),
    ],
)

print(f"Batch_size: {str(bs)}")
loss = model.evaluate(X_test, y_test, verbose=0)

Epoch 1/50
154/154 [==============================] - 3s 11ms/step - loss: 0.9756 - val_loss: 0.6586
Epoch 2/50
154/154 [==============================] - 2s 10ms/step - loss: 0.6748 - val_loss: 0.6312
Epoch 3/50
154/154 [==============================] - 1s 10ms/step - loss: 0.8387 - val_loss: 0.6589
Epoch 4/50
154/154 [==============================] - 2s 10ms/step - loss: 0.6600 - val_loss: 0.6468
Epoch 5/50
154/154 [==============================] - 1s 10ms/step - loss: 0.6476 - val_loss: 0.6463
Epoch 6/50
154/154 [==============================] - 2s 10ms/step - loss: 0.6522 - val_loss: 0.6460
Epoch 7/50
154/154 [==============================] - 2s 10ms/step - loss: 0.6478 - val_loss: 0.6458
Epoch 8/50
154/154 [==============================] - 2s 10ms/step - loss: 0.6466 - val_loss: 0.6457
Epoch 9/50
154/154 [==============================] - 2s 10ms/step - loss: 0.6470 - val_loss: 0.6457
Epoch 10/50
154/154 [==============================] - 2s 10ms/step - loss: 0.6516 - val_lo

In [64]:
pred = model.predict(X_test)

In [65]:
pred

array([[0.20459574],
       [0.3543476 ],
       [0.36457402],
       ...,
       [0.28134596],
       [0.36416388],
       [0.3762661 ]], dtype=float32)

In [68]:
y_test

array([[0],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=uint8)